In [10]:
import os
import re
import csv
import math
import time
import json
import random
import finnhub
import datasets
import newspaper
import nltk
nltk.download('punkt')
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime
from gnews import GNews
from dotenv import load_dotenv
from collections import defaultdict
from datasets import Dataset
from openai import OpenAI

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Ingest Data

In [11]:
START_DATE = "2023-01-01"
END_DATE = "2024-01-16"

DATA_DIR = f"./train_data/{START_DATE}_{END_DATE}"
os.makedirs(DATA_DIR, exist_ok=True)

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RAPID_API_KEY = os.getenv("RAPID_API_KEY")

In [ ]:
returns = get_returns('CBA.AX')
print(returns)

In [ ]:
#google_news = GNews(language='en', country='AU', start_date=(2023,12,15), end_date=(2024,1,16))

# google_news.country = 'Australia'  # News from a specific country 
# google_news.language = 'english'  # News in a specific language
#google_news.start_date = (2020, 1, 1) # Search from 1st Jan 2020
# google_news.end_date = (2020, 3, 1) # Search until 1st March 2020
#google_news.period = '7d'  # News from last 7 days
#google_news.max_results = 10  # number of responses across a keyword
# #google_news.exclude_websites = ['yahoo.com', 'cnn.com']  # Exclude news from specific website i.e Yahoo.com and CNN.com

CBA = GNews(language='en', country='AU', start_date=(2023, 12, 15), end_date=(2024, 1, 16)).get_news('CBA')
# CBA = google_news.get_news('CBA')
print(CBA)


In [ ]:
from datetime import datetime
#print(CBA[1])

date_string = CBA[1]['published date']
formatted_date_object = datetime.strptime(date_string, '%a, %d %b %Y %H:%M:%S %Z').date()
formatted_date_tuple = (formatted_date_object.year,formatted_date_object.month,formatted_date_object.day)
print(formatted_date_tuple)

In [19]:
def format_date_tuple(date: datetime) -> tuple:
    return (date.year,date.month,date.day)


def news_summarizer(url) -> str:
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 newspaper/0.2.8 Edg/120.0.0.0'
    config = newspaper.Config()
    config.browser_user_agent = user_agent
    try:
        article = newspaper.Article(url="%s" % url, config=config)
        article.download()
        article.parse()
        article.nlp()
    except Exception as error:
        print(f"An error occurred while fetching the article: {error}")
        return None
    
    return article.summary.replace('\n',' ')
    #return article.summary[:article.summary.find('[')].replace('\n',' ')


def bin_mapping(ret):
    
    up_down = 'U' if ret >= 0 else 'D'
    integer = math.ceil(abs(100 * ret))
    
    return up_down + (str(integer) if integer <= 5 else '5+')


def get_returns(stock_symbol):
    
    # Download historical stock data
    stock_data = yf.download(stock_symbol, start=START_DATE, end=END_DATE)
    
    weekly_data = stock_data['Adj Close'].resample('W').ffill()
    weekly_returns = weekly_data.pct_change()[1:]
    weekly_start_prices = weekly_data[:-1]
    weekly_end_prices = weekly_data[1:]

    weekly_data = pd.DataFrame({
        'Start Date': weekly_start_prices.index,
        'Start Price': weekly_start_prices.values,
        'End Date': weekly_end_prices.index,
        'End Price': weekly_end_prices.values,
        'Weekly Returns': weekly_returns.values
    })
    
    weekly_data['Bin Label'] = weekly_data['Weekly Returns'].map(bin_mapping)

    return weekly_data


def get_news(symbol:str, data:pd.DataFrame) -> pd.DataFrame:
    news_list = []

    for end_date, row in data.iterrows():
        start_date= format_date_tuple(row['Start Date'])
        end_date = format_date_tuple(row['End Date'])
        print(symbol, ': ', start_date, ' - ', end_date)
        time.sleep(1) # control qpm

        weekly_news = GNews(language='en', country='AU', start_date=start_date, end_date=end_date).get_news(symbol)
        weekly_news = [
            {
                "date": datetime.strptime(n['published date'], '%a, %d %b %Y %H:%M:%S %Z').strftime('%Y%m%d%H%M%S'),
                "headline": n['title'],
                "summary": news_summarizer(n['url']),
            } for n in weekly_news
        ]
        weekly_news.sort(key=lambda x: x['date'])
        news_list.append(json.dumps(weekly_news))

    data['News'] = news_list
    
    return data    
    
    
def prepare_data_for_company(symbol):
    
    data = get_returns(symbol)
    data = get_news(symbol, data)    
    data.to_csv(f"{DATA_DIR}/{symbol}_{START_DATE}_{END_DATE}.csv")
    
    return data
    

In [17]:
ASX_20 = [
            'REA.AX']
#'CBA.AX', 'CSL.AX', 'BHP.AX', 'WBC.AX', 'NAB.AX', 'ANZ.AX', 'WOW.AX', 'WES.AX', 'MQG.AX','RIO.AX','TLS.AX', 'FMG.AX', 'GMG.AX', 'NEM.AX', 'JHX.AX', 'STO.AX','ALL.AX', 'WTC.AX', 'WDS.AX', 

In [15]:
CBA = GNews(language='en', country='AU', start_date=(2023, 12, 15), end_date=(2024, 1, 16)).get_news('CBA')
print(news_summarizer(CBA[4]['url']))

After slipping for three months in the second half of 2023, Commonwealth Bank’s mortgage business has rebounded, while the other big four banks keep growing, according to new data from the prudential regulator.


In [16]:
print(news_summarizer(CBA[1]['url']))

The Commonwealth Bank of Australia’s (CBA) latest Home Buying Index has revealed a sharp decline of 31.1 per cent per month in December in seasonally adjusted terms, leaving the index at 64.2. The index deteriorated on an annual basis as well, dropping 40 per cent year on year from the 9.9 per cent annual decline recorded in October 2023. CBA also predicted that home prices are to continue to rise throughout 2024, however, at a slower pace than the 9.3 per cent recorded in 2023. CBA senior economist Belinda Allen said that household budgets were “undoubtedly constrained with rate rises leading to a weakening of consumer spending”. [RELATED: ‘High chance’ of RBA achieving its goals in 2024: CBA]


In [20]:
# check article resources dir
dir_path = 'C:/Users/LENOVO/AppData/Local/Temp/.newspaper_scraper/article_resources'

# Check if the directory exists
if os.path.exists(dir_path):
    # Print a message if the directory already exists
    print("The directory already exists")
else:
    # Create the directory if it does not exist
    os.makedirs(dir_path)
    # Print a message if the directory is created
    print("The directory is created")


# prepare data
for symbol in ASX_20:
    prepare_data_for_company(symbol)

The directory already exists
[*********************100%***********************]  1 of 1 completed
REA.AX :  (2023, 1, 8)  -  (2023, 1, 15)
An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.theirishworld.com/peter-kavanagh/ on URL https://news.google.com/rss/articles/CBMiLWh0dHBzOi8vd3d3LnRoZWlyaXNod29ybGQuY29tL3BldGVyLWthdmFuYWdoL9IBAA?oc=5&hl=en-AU&gl=AU&ceid=AU:en


KeyboardInterrupt: 

In [ ]:
def get_last_processed_symbol():
    try:
        with open('last_processed_symbol.txt', 'r') as file:
            return file.read().strip()
    except FileNotFoundError:
        return None

def update_last_processed_symbol(symbol):
    with open('last_processed_symbol.txt', 'w') as file:
        file.write(symbol)

last_processed_symbol = get_last_processed_symbol()

for symbol in ASX_20:
    if symbol == last_processed_symbol:
        last_processed_symbol = None  # Start from the next symbol
    elif last_processed_symbol is None:
        prepare_data_for_company(symbol)
        update_last_processed_symbol(symbol)

# Generate Target Column using GPT4

## Generate user's prompt

In [ ]:
# User prompt

def get_profile(symbol:str) -> dict:

    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-profile"

    querystring = {"symbol":symbol,"region":"AU","lang":"en-AU"}

    headers = {
        "X-RapidAPI-Key": RAPID_API_KEY,
        "X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    profile = response.json()['quoteSummary']['result'][0]['summaryProfile']['longBusinessSummary']

    return profile


def get_company_prompt(symbol):
    
    profile = get_profile(symbol=symbol)

    company_introduction = f"[Company Introduction]: {profile}"
    
    return company_introduction


def get_prompt_by_row(symbol, row):

    start_date = row['Start Date'] if isinstance(row['Start Date'], str) else row['Start Date'].strftime('%Y-%m-%d')
    end_date = row['End Date'] if isinstance(row['End Date'], str) else row['End Date'].strftime('%Y-%m-%d')
    term = 'increased' if row['End Price'] > row['Start Price'] else 'decreased'
    head = "From {} to {}, {}'s stock price {} from {:.2f} to {:.2f}. Company news during this period are listed below:\n\n".format(
        start_date, end_date, symbol, term, row['Start Price'], row['End Price'])
    
    news = json.loads(row["News"])
    news = ["[Headline]: {}\n[Summary]: {}\n".format(
        n['headline'], n['summary']) for n in news if n['date'][:8] <= end_date.replace('-', '') and \
        not n['summary'].startswith("Looking for stock market analysis and research with proves results?")]
 
    return head, news


def sample_news(news, k=5):
    
    return [news[i] for i in sorted(random.sample(range(len(news)), k))]


def map_bin_label(bin_lb):
    
    lb = bin_lb.replace('U', 'up by ')
    lb = lb.replace('D', 'down by ')
    lb = lb.replace('1', '0-1%')
    lb = lb.replace('2', '1-2%')
    lb = lb.replace('3', '2-3%')
    lb = lb.replace('4', '3-4%')
    if lb.endswith('+'):
        lb = lb.replace('5+', 'more than 5%')
#         lb = lb.replace('5+', '5+%')
    else:
        lb = lb.replace('5', '4-5%')
    
    return lb


def get_all_prompts(symbol, min_past_weeks=1, max_past_weeks=3):
    
    df = pd.read_csv(f'{DATA_DIR}/{symbol}_{START_DATE}_{END_DATE}.csv')
    
    company_prompt = get_company_prompt(symbol)

    prev_rows = []
    all_prompts = []

    for row_idx, row in df.iterrows():

        prompt = ""
        if len(prev_rows) >= min_past_weeks:
            idx = min(random.choice(range(min_past_weeks, max_past_weeks+1)), len(prev_rows))
            for i in range(-idx, 0):
                # Add Price Movement (Head)
                prompt += "\n" + prev_rows[i][0]
                # Add News of previous weeks
                sampled_news = sample_news(
                    prev_rows[i][1],
                    min(5, len(prev_rows[i][1]))
                )
                if sampled_news:
                    prompt += "\n".join(sampled_news)
                else:
                    prompt += "No relative news reported."

        head, news = get_prompt_by_row(symbol, row)

        prev_rows.append((head, news))
        if len(prev_rows) > max_past_weeks:
            prev_rows.pop(0)  

        if not prompt:
            continue

        prediction = map_bin_label(row['Bin Label'])
        
        prompt = company_prompt + '\n' + prompt
        prompt += f"\n\nBased on all the information before {row['Start Date']}, let's first analyze the positive developments and potential concerns for {symbol}. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. " \
            f"Then let's assume your prediction for next week ({row['Start Date']} to {row['End Date']}) is {prediction}. Provide a summary analysis to support your prediction. The prediction result need to be inferred from your analysis at the end, and thus not appearing as a foundational factor of your analysis."

        all_prompts.append(prompt.strip())
    
    return all_prompts

## Generate system prompt

In [ ]:
# System prompt

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"


SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. " \
    "Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]:\n...\n"

print(SYSTEM_PROMPT)

# prompts = get_all_prompts("AAPL", 1, 3)
prompts = get_all_prompts("CBA.AX", 1, 4)

print(prompts[0])

## Use GPT-4 for Financial Analysis

In [ ]:
def append_to_csv(filename, input_data, output_data):
    
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([input_data, output_data])

        
def initialize_csv(filename):
    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["prompt", "answer"])


def query_gpt4(symbol_list, min_past_weeks=1, max_past_weeks=3):

    for symbol in symbol_list:
        
        csv_file = f'{DATA_DIR}/{symbol}_{START_DATE}_{END_DATE}_gpt-4.csv'
                          
        if not os.path.exists(csv_file):
            initialize_csv(csv_file)
            pre_done = 0
        else:
            df = pd.read_csv(csv_file)
            pre_done = len(df)

        prompts = get_all_prompts(symbol, min_past_weeks, max_past_weeks)

        for i, prompt in enumerate(prompts):
            
            if i < pre_done:
                continue

            print(f"{symbol} - {i}")
            
            cnt = 0
            while cnt < 5:
                try:
                    completion = client.chat.completions.create(
                        model="gpt-4",
                        messages=[
                            {"role": "system", "content": SYSTEM_PROMPT},
                            {"role": "user", "content": prompt}
                          ]
                    )
                    break    
                except Exception:
                    cnt += 1
                    print(f'retry cnt {cnt}')
            
            answer = completion.choices[0].message.content if cnt < 5 else ""
            append_to_csv(csv_file, prompt, answer)

In [ ]:
query_gpt4(ASX_20, 1, 4)

## Transform into Llama2 Training Format

In [ ]:
def gpt4_to_llama(symbol):
    
    csv_file = f'{DATA_DIR}/{symbol}_{START_DATE}_{END_DATE}_gpt-4.csv'
    
    df = pd.read_csv(csv_file)
    
    prompts, answers, periods, labels = [], [], [], []
    
    for i, row in df.iterrows():
        
        prompt, answer = row['prompt'], row['answer']
        
        res = re.search(r"Then let's assume your prediction for next week \((.*)\) is ((:?up|down) by .*%).", prompt)
        
        period, label = res.group(1), res.group(2)
#         label = label.replace('more than 5', '5+')
        
        prompt = re.sub(
            r"Then let's assume your prediction for next week \((.*)\) is (up|down) by ((:?.*)%). Provide a summary analysis to support your prediction. The prediction result need to be inferred from your analysis at the end, and thus not appearing as a foundational factor of your analysis.", 
            f"Then make your prediction of the {symbol} stock price movement for next week ({period}). Provide a summary analysis to support your prediction.",
            prompt
        )
        try:
            answer = re.sub(
                r"\[Prediction & Analysis\]:\s*",
                f"[Prediction & Analysis]:\nPrediction: {label.capitalize()}\nAnalysis: ",
                answer
            )
        except Exception:
            print(symbol, i)
            print(label)
            print(answer)
            continue
            
        new_system_prompt = SYSTEM_PROMPT.replace(':\n...', '\nPrediction: ...\nAnalysis: ...')
#         new_system_prompt = SYSTEM_PROMPT.replace(':\n...', '\nPrediction: {Up|Down} by {1-2|2-3|3-4|4-5|5+}%\nAnalysis: ...')
        
        prompt = B_INST + B_SYS + new_system_prompt + E_SYS + prompt + E_INST
        
        prompts.append(prompt)
        answers.append(answer)
        periods.append(period)
        labels.append(label)
        
    return {
        "prompt": prompts,
        "answer": answers,
        "period": periods,
        "label": labels,
    }


def create_dataset(symbol_list, train_ratio=0.8, with_basics=True):

    train_dataset_list = []
    test_dataset_list = []

    for symbol in symbol_list:

        data_dict = gpt4_to_llama(symbol, with_basics)
#         print(data_dict['prompt'][-1])
#         print(data_dict['answer'][-1])
        symbols = [symbol] * len(data_dict['label'])
        data_dict.update({"symbol": symbols})

        dataset = Dataset.from_dict(data_dict)
        train_size = round(train_ratio * len(dataset))

        train_dataset_list.append(dataset.select(range(train_size)))
        test_dataset_list.append(dataset.select(range(train_size, len(dataset))))

    train_dataset = datasets.concatenate_datasets(train_dataset_list)
    test_dataset = datasets.concatenate_datasets(test_dataset_list)

    dataset = datasets.DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })
    
    return dataset